In [1]:
#СОЗДАЕТ И ОБУЧАЕТ МОДЕЛЬ (MLP) ПО МОДЕЛИ СОЗДАЕТ МАССИВ МАКСИМУМОВ
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import mediapipe as mp
import torch.nn.functional as F
import cv2
import time
import sys
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [14]:
hand = 'l'

In [15]:
path_to_model = "D:/BMM/models/" + hand + "_model.pth" #СЮДА СОХРАНИТ МОДЕЛЬ
path_to_dataset = "D:/BMM/datasets/" + hand + "_dataset/" #ЗДЕСЬ БУДЕТ ИСКАТЬ ДАТАСЕТЫ (r_dataset / l_dataset)
path_to_maxmass = "D:/BMM/maxmasses/" + hand + "_maxmass" #СЮДА СОХРАНИТ МАССИВ МАКСИМУМОВ

In [16]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [17]:
data, temp = [],[]
gests = os.listdir(path_to_dataset)
for gest in gests:
    data = []
    print(gest)
    f = open(path_to_dataset + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
if hand == 'l':
    data = np.array(temp ,dtype = 'object') 
elif hand == 'r':  
    data = np.array(temp)
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

l_kulak.txt
l_ladon.txt


In [19]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

In [22]:
for epoch in range(20):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss/len(dataloader))
print('Finished Training')

0.2978659213210146
0.012931950044788657
0.0030998466776624794
0.0013676149500829596
0.0008013079073169225
0.0005361589919918934
0.00038218901167893485
0.00029067978461431006
0.0002113910431078347
0.000178242727391383
0.00013982081899095128
0.00011483630696025174
0.00010053713965986851
8.736237065484543e-05
7.54683881221729e-05
8.092936620833205e-05
6.214754150624642e-05
6.172036760218029e-05
5.946989230078688e-05
5.493322559564221e-05
Finished Training


In [23]:
torch.save(net,path_to_model)

In [24]:
tmaxmass =[[0,0],[0,0]] # ДЛИНА МАССИВА ДАЛЖНА БЫТЬ РАВНА КОЛИЧЕСТВУ ЖЕСТОВ
#tmaxmass = [[1,2],[3,4],[5,6],[7,8]]
for epoch in range(15):
    for samples, labels in dataloader:
        for j in samples:
            pred = net(j).tolist()
            mx = max(pred)
            mxid = pred.index(mx)
            tmaxmass[mxid][0] = tmaxmass[mxid][0] + mx
            tmaxmass[mxid][1] = tmaxmass[mxid][1] + 1
maxmass = [tmaxmass[i][0] / tmaxmass[i][1] for i in range(len(tmaxmass)) ]
print('СРЕДНИЕ МАКСИМУМЫ КАЖДОГО ЖЕСТА', maxmass)
gests = [i[:-4] for i in gests]

СРЕДНИЕ МАКСИМУМЫ КАЖДОГО ЖЕСТА [5.045858645819098, 7.284374559794987]


In [25]:
maxmass = np.array(maxmass)
np.set_printoptions(threshold=sys.maxsize)
np.save(path_to_maxmass, maxmass)